# Complex TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando.vis import mpl

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
from datetime import datetime
today = datetime.today()
# dd/mm/YY
d4 = today.strftime("%d-%b-%Y-%Hh:%Mm")

### Auxiliary functions

In [3]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [4]:
model_dir = 'data/thick_inter_body/model.pickle'
data_dir = 'data/thick_inter_body/data.pickle'
#grid_dir = '../grids/flightlines_grid_2100pts_100km2_-150m.pickle'

In [5]:
with open(model_dir) as w:
        model = pickle.load(w)

In [6]:
with open(data_dir) as w:
        data = pickle.load(w)

In [7]:
dobs = data['tfa_obs']
xp = data['grid'][0]
yp = data['grid'][1]
zp = data['grid'][2]
N = data['grid'][3]

### Parameters of the initial model

In [8]:
M = 20 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

int_min = 7.
int_max = 16.
intensity = np.linspace(int_min, int_max, 10)

# depth to the top, thickness and radius
z0_min = 0.
z0_max = 360.
z0 = np.linspace(z0_min, z0_max, 10)
dz = 650.
r = 800.
x0 = -300.
y0 = 300.

# main field
inc, dec = data['main_field']

In [9]:
z0

array([  0.,  40.,  80., 120., 160., 200., 240., 280., 320., 360.])

In [10]:
intensity

array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.])

### Limits

In [11]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -4000.
x0max = 4000.
y0min = -4000.
y0max = 4000.
dzmin = 200.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [12]:
# variation for derivatives
deltax = 0.01*np.max(100.)
deltay = 0.01*np.max(100.)
deltar = 0.01*np.max(100.)
deltaz = 0.01*np.max(100.)

### Outcropping parameters

In [13]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

In [16]:
# output of inversion
inversion = dict()

### Regularization parameters

In [17]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-5  # adjacent radial distances within each prism
a2 = 1.0e-4   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4    # vertically adjacent origins
a6 = 1.0e-8   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-7     # zero order Tikhonov on thickness of each prism

In [18]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [19]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [20]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [21]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
#inversion['initial_estimate'] = model0
#inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity

### Inversion

In [22]:
inversion_results = []
for j, z in enumerate(z0):
    for k, i in enumerate(intensity):
        alpha = np.array([a1, a2, a3, a4, a5, a6, a7])
        print 'inversion: %d  top: %d  intensity: %d' % (j*z0.size + k, z, i)
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([m_est, phi_list, model_list, dobs - d_fit])

inversion: 0  top: 0  intensity: 7
it:  0   it_marq:  0   lambda: 1e+01   init obj.: 4.02387e+04  fin obj.: 2.72423e+04
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 2.72423e+04  fin obj.: 1.62725e+04
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 1.62725e+04  fin obj.: 4.58560e+04
it:  2   it_marq:  1   lambda: 1e+00   init obj.: 1.62725e+04  fin obj.: 1.45814e+04
it:  3   it_marq:  0   lambda: 1e-01   init obj.: 1.45814e+04  fin obj.: 2.34021e+04
it:  3   it_marq:  1   lambda: 1e+00   init obj.: 1.45814e+04  fin obj.: 1.25076e+04
it:  4   it_marq:  0   lambda: 1e-01   init obj.: 1.25076e+04  fin obj.: 2.34606e+04
it:  4   it_marq:  1   lambda: 1e+00   init obj.: 1.25076e+04  fin obj.: 8.64222e+03
it:  5   it_marq:  0   lambda: 1e-01   init obj.: 8.64222e+03  fin obj.: 1.53838e+04
it:  5   it_marq:  1   lambda: 1e+00   init obj.: 8.64222e+03  fin obj.: 1.16982e+04
it:  5   it_marq:  2   lambda: 1e+01   init obj.: 8.64222e+03  fin obj.: 8.28961e+03
it:  6   it_marq:  0   lambda:

# Results

In [23]:
inversion['results'] = inversion_results

### Folder to save the results

In [14]:
mypath = data_dir[:-11]+'l2_tfa_inversion/multiple-'+d4 #default folder name
# mypath = data_dir[:-11]+'l2_tfa_inversion/name_folder' #defined folder name
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [24]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)

In [25]:
mypath

'data/thick_inter_body/l2_tfa_inversion/multiple-10-Jul-2020-22h:05m'